# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [17]:
import os

# Set your preferred name
USER = "Jinglin"
# Set the data and output directories
DATA_ROOT = '../../DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Testicular Cancer'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
# NORMALIZE_GENE = False
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [3]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Bladder_Cancer_(BLCA)',
 '.DS_Store',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Stomach_Cancer_(STAD)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [4]:
trait_subdir = "TCGA_Testicular_Cancer_(TGCT)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.TGCT.sampleMap_TGCT_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.TGCT.sampleMap_HiSeqV2_PANCAN.gz')

In [5]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [6]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [7]:
clinical_data_cols[:10]

['_INTEGRATION',
 '_PATIENT',
 '_cohort',
 '_primary_disease',
 '_primary_site',
 'age_at_initial_pathologic_diagnosis',
 'bcr_followup_barcode',
 'bcr_patient_barcode',
 'bcr_sample_barcode',
 'bilateral_diagnosis_timing_type']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [8]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'bilateral_diagnosis_timing_type', 'clinical_M', 'clinical_N', 'clinical_T', 'clinical_stage', 'days_to_bilateral_tumor_dx', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatmen

In [9]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis',
                      'days_to_birth']
candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [10]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [31.0, 38.0, 28.0, 30.0, 28.0],
 'days_to_birth': [-11325.0, -13964.0, -10511.0, -10983.0, -10281.0]}

In [11]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [12]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'MALE', 'MALE', 'MALE']}

In [13]:
gender_col = 'gender'

In [14]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [18]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

15 input query terms found dup hits:	[('GTF2IP1', 2), ('RBMY1A3P', 3), ('HERC2P2', 3), ('WASH3P', 3), ('NUDT9P1', 2), ('SNORD104', 2), ('
154 input query terms found no hit:	['C16orf13', 'C16orf11', 'LOC100272146', 'LOC339240', 'NACAP1', 'LOC441204', 'KLRA1', 'FAM183A', 'FA
9 input query terms found dup hits:	[('SUGT1P1', 2), ('SNORD127', 2), ('SNORA62', 4), ('IFITM4P', 8), ('HLA-DRB6', 2), ('FUNDC2P2', 2), 
190 input query terms found no hit:	['NARFL', 'NFKBIL2', 'LOC150197', 'TMEM84', 'LOC162632', 'PPPDE1', 'PPPDE2', 'C1orf38', 'C1orf31', '
20 input query terms found dup hits:	[('PIP5K1P1', 2), ('SNORA40', 2), ('SNORD116-24', 2), ('SNORD116-25', 2), ('SNORD116-23', 2), ('SNOR
149 input query terms found no hit:	['FAM153C', 'C9orf167', 'CLK2P', 'CCDC76', 'CCDC75', 'CCDC72', 'HIST3H2BB', 'PRAC', 'LOC285780', 'LO
18 input query terms found dup hits:	[('SNORD58C', 2), ('UOX', 2), ('UBE2Q2P1', 3), ('PPP4R1L', 2), ('SNORD63', 3), ('SNORD116-22', 2), (
158 input query terms found no hit:	['

In [20]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Testicular Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-2G-AAEW-01,1,31.0,1.0,-1.247192,1.370174,9.255665,0.322728,-0.485378,-0.61291,1.315306,...,3.498714,-0.086682,4.207622,-0.237217,1.597767,0.23401,2.696223,3.097773,-1.468595,-0.043933
TCGA-2G-AAEX-01,1,38.0,1.0,-1.743692,1.814274,9.680665,0.636028,0.108222,-1.26731,1.206606,...,3.790214,0.487618,5.504822,0.846083,0.797867,0.02141,2.778623,0.329973,-1.447895,1.108067
TCGA-2G-AAF1-01,1,28.0,1.0,-1.397192,1.187274,9.598765,0.838328,-0.096978,-1.04111,1.560006,...,3.421714,-0.086682,4.813122,-0.005417,1.629067,0.64101,2.337123,2.008273,-1.256195,0.153367
TCGA-2G-AAF4-01,1,30.0,1.0,-0.994992,1.597874,9.971765,0.339728,-0.098878,-1.05091,0.858706,...,4.097814,-0.086682,4.213022,2.064083,1.162567,0.53161,3.066823,-0.735627,-1.617995,-0.710533
TCGA-2G-AAF6-01,1,28.0,1.0,-0.498792,2.274374,8.821565,0.247628,0.408722,-0.65491,1.862706,...,4.072914,-0.086682,5.124822,1.812383,0.734067,-0.88259,2.702423,-1.043427,-1.584895,-0.367333


In [21]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 139 samples.


In [22]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Testicular Cancer', the least common label is '1' with 139 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Testicular Cancer' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 26.0
  50% (Median): 31.0
  75%: 37.0
Min: 14.0
Max: 67.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 139 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



True

In [23]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [24]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Jinglin/Testicular-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [25]:
dataset = 'GEO'
trait_subdir = "Testicular-Cancer"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE3921',
 'GSE1818',
 'GSE59520',
 'GSE12630',
 'GSE62523',
 'GSE28094',
 'GSE42647']

Repeat the below steps for all the accession numbers

In [104]:
cohort = accession_num = "GSE42647"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('../../DATA/GEO/Testicular-Cancer/GSE42647/GSE42647_family.soft.gz',
 '../../DATA/GEO/Testicular-Cancer/GSE42647/GSE42647-GPL6947_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [105]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Acute hypersensitivity of pluripotent testicular cancer-derived embryonal carcinoma to low-dose 5-aza deoxycytidine"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"


In [106]:
clinical_data.head()

,!Sample_geo_accession,GSM1047246,GSM1047247,GSM1047248,GSM1047249,GSM1047250,GSM1047251,GSM1047252,GSM1047253,GSM1047254,GSM1047255,GSM1047256,GSM1047257
0,!Sample_characteristics_ch1,cell line: NT2/D1-R1,cell line: NT2/D1-R1,cell line: NT2/D1-R1,cell line: NT2/D1-R1,cell line: NT2/D1-R1,cell line: NT2/D1-R1,cell line: NT2/D1-R1,cell line: NT2/D1-R1,cell line: NT2/D1-R1,cell line: NT2/D1-R1,cell line: NT2/D1-R1,cell line: NT2/D1-R1
1,!Sample_characteristics_ch1,cell type: human ebryonal carcinoma,cell type: human ebryonal carcinoma,cell type: human ebryonal carcinoma,cell type: human ebryonal carcinoma,cell type: human ebryonal carcinoma,cell type: human ebryonal carcinoma,cell type: human ebryonal carcinoma,cell type: human ebryonal carcinoma,cell type: human ebryonal carcinoma,cell type: human ebryonal carcinoma,cell type: human ebryonal carcinoma,cell type: human ebryonal carcinoma


In [107]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['cell line: NT2/D1-R1'], 1: ['cell type: human ebryonal carcinoma']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [108]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.
   NOTE: Function "convert_trait" should return 1 or 0.
   
Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Testicular Cancer\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Testicular Cancer\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The 

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [109]:
# is_gene_availabe = True
is_gene_availabe = False
trait_row_id = 1
trait_type = 'binary'

age_row_id = None
# age_row_id = 1

gender_row_id = None
# gender_row_id = 0

In [110]:
is_available = is_gene_availabe and (trait_row_id is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

This cohort is not usable. Please skip the following steps and jump to the next accession number.


In [89]:
def convert_trait(value):
    if 'testis cancer' in value or 'testis tumor tissue' in value:
        return 1
    else:
        return 0

def convert_age(value):
    try:
        if value.startswith('age: '):
            return float(value.split(': ')[1])
        else:
            return None
    except ValueError:
        return None

    
def convert_gender(value):
    if value == 'gender: male':
        return 1
    elif value == 'gender: female':
        return 0
    else:
        return None


In [90]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row_id, age_row_id, gender_row_id,
                                                      convert_trait, convert_age, convert_gender)
selected_clinical_data.head()

,GSM694425,GSM694426,GSM694427,GSM694428,GSM694429,GSM694430,GSM694431,GSM694432,GSM694433,GSM694434,...,GSM696043,GSM696044,GSM696045,GSM696046,GSM696047,GSM696048,GSM696049,GSM696050,GSM696051,GSM696052
Testicular Cancer,0,0,0.0,0,0,0.0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0.0
Age,None,None,65.0,None,None,79.0,None,None,51.0,None,...,None,None,None,None,None,None,None,None,None,56.0
Gender,None,None,1.0,None,None,1.0,None,None,1.0,None,...,None,None,None,None,None,None,None,None,None,0.0


### Genetic data preprocessing and final filtering

In [91]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,ID,GSM694425,GSM694426,GSM694427,GSM694428,GSM694429,GSM694430,GSM694431,GSM694432,GSM694433,...,GSM696043,GSM696044,GSM696045,GSM696046,GSM696047,GSM696048,GSM696049,GSM696050,GSM696051,GSM696052
0,AATK_E63_R,0.48353,0.23711,0.85411,0.50129,0.82760,0.59996,0.68489,0.90238,0.92263,...,0.09186,0.91331,0.93600,0.16329,0.91098,0.34287,0.41839,0.32744,0.68380,0.88365
1,AATK_P519_R,0.89311,0.84668,0.70404,0.66548,0.89701,0.64044,0.88225,0.90956,0.92516,...,0.89194,0.92933,0.92402,0.05399,0.93193,0.22968,0.30236,0.20211,0.80366,0.91518
2,AATK_P709_R,0.91335,0.30281,0.37084,0.26867,0.26395,0.38570,0.33783,0.78834,0.87449,...,0.77597,0.76322,0.84806,0.02906,0.69019,0.07539,0.06704,0.12400,0.59265,0.82164
3,ABCA1_E120_R,0.03472,0.07660,0.03341,0.01977,0.03680,0.03003,0.03845,0.19101,0.02931,...,0.26583,0.42989,0.03674,0.02782,0.03731,0.01852,0.02787,0.02626,0.04036,0.02370
4,ABCA1_P45_F,0.07617,0.05157,0.02845,0.03297,0.06749,0.05253,0.04280,0.04870,0.03439,...,0.70815,0.11157,0.14586,0.04181,0.03450,0.03162,0.09569,0.03407,0.03279,0.01979


In [92]:
gene_row_ids = genetic_data['ID'][:20].tolist()
gene_row_ids

['AATK_E63_R',
 'AATK_P519_R',
 'AATK_P709_R',
 'ABCA1_E120_R',
 'ABCA1_P45_F',
 'ABCB4_E429_F',
 'ABCB4_P51_F',
 'ABCB4_P892_F',
 'ABCC2_E16_R',
 'ABCC2_P88_F',
 'ABCC5_P444_F',
 'ABCG2_P178_R',
 'ABCG2_P310_R',
 'ABL1_P53_F',
 'ABL2_P459_R',
 'ABO_E110_F',
 'ABO_P312_F',
 'ACTG2_E98_R',
 'ACTG2_P346_F',
 'ACTG2_P455_R']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [93]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['AATK_E63_R', 'AATK_P519_R', 'AATK_P709_R', 'ABCA1_E120_R', 'ABCA1_P45_F', 'ABCB4_E429_F', 'ABCB4_P51_F', 'ABCB4_P892_F', 'ABCC2_E16_R', 'ABCC2_P88_F', 'ABCC5_P444_F', 'ABCG2_P178_R', 'ABCG2_P310_R', 'ABL1_P53_F', 'ABL2_P459_R', 'ABO_E110_F', 'ABO_P312_F', 'ACTG2_E98_R', 'ACTG2_P346_F', 'ACTG2_P455_R']\n"

If not required, jump directly to the gene normalization step

In [94]:
requires_gene_mapping = True

In [95]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)[0]
    gene_annotation_summary = preview_df(gene_annotation)
    gene_annotation_summary

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [96]:
# if requires_gene_mapping:
f'''
As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
probe_id_key = key_name1
gene_symb_key = key_name2

Gene annotation dictionary:
{gene_annotation_summary}
'''

"\nAs a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:\nprobe_id_key = key_name1\ngene_symb_key = key_name2\n\nGene annotation dictionary:\n{'ID': ['AATK_E63_R', 'AATK_P519_R', 'AATK_P709_R', 'ABCA1_E120_R', 'ABCA1_P45_F'], 'Gid': ['89041906', '89041906', '89041906', '21536375', '21536375'], 'GB_ACC': ['XM_927215.1', 'XM_927215.1', 'XM_927215.1', 'NM_005502.2', 'NM_005502.2'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Symbol': ['AATK', 'AATK', 'AATK', 'ABCA1', 'ABCA1'], 'Gene_ID': ['9625', '9625', '9625', '19', '19'], 'Chromosome': ['17', '17', '17', '9', '9'], 'REFSEQ': [36.1, 36.1, 36.1, 36.1, 36.1], 'CpG_Coordinate': [76709831.0, 76710413.0, 76710603.0, 106730137.0, 106730302.0], 'Dist_to_TSS': ['63', '-519', '-709', '120', '-45'], 'CpG

In [97]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [98]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

1 input query terms found dup hits:	[('LY6G6E', 8)]
47 input query terms found no hit:	['BIRC4', 'CARD15', 'CSPG2', 'CTGF', 'DBC1', 'DKFZP564O0823', 'DNMT2', 'EMR3', 'EVI1', 'FAT', 'FVT1'


In [99]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [100]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1261 samples.


In [101]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Testicular Cancer', the least common label is '0.0' with 1261 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Testicular Cancer' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 47.0
  50% (Median): 62.0
  75%: 72.0
Min: 0.0
Max: 106.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 617 occurrences. This represents 48.93% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



True

In [102]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [103]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [111]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

In [112]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note


In [155]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [195]:
merged_data.head()

,Thyroid Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
0,1,74,0,-0.767592,-1.742926,-0.531035,-0.067972,0.265922,-0.13531,0.440006,...,-0.748878,-1.018517,0.205567,-1.28139,0.395523,1.735414,0.945273,-0.186695,4.024767,-0.273612
1,1,65,1,1.766308,0.273374,-0.531035,0.790028,1.209322,0.01229,0.704606,...,0.125322,1.568883,0.626367,2.38101,0.275823,1.820014,2.140073,1.380405,1.257367,-0.174312
2,1,55,1,0.986808,-2.165726,-0.050935,0.051428,0.979522,0.48229,0.090106,...,-0.748878,-0.747417,-0.931133,-1.28139,-0.222077,0.293414,0.725173,-0.254095,0.530667,-0.430812
3,1,57,1,0.767208,-1.623626,-0.531035,0.182228,0.541622,0.65889,0.647206,...,-0.748878,-1.587117,-1.292433,-1.28139,-0.602177,0.134114,0.894473,-0.380395,-2.502833,0.360588
4,1,33,0,-0.287292,-1.766626,-0.531035,0.059628,0.920522,0.21849,0.336106,...,-0.413778,1.111583,1.158867,-1.28139,0.003723,1.240514,0.924373,-0.143495,2.613367,-0.030012


In [198]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [199]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [200]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [201]:
if has_batch_effect:
    model_constructor = VariableSelection
    model_params = {}
else:
    model_constructor = Lasso
    model_params = {'alpha': 1.0, 'random_state': 42}

In [202]:
cv_mean, cv_std = cross_validation(X, Y, Z, model_constructor, model_params, target_type=trait_type)

/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


The cross-validation accuracy is 83.16% ± 5.64%


In [203]:
# Train regression model on the whole dataset to identify significant genes
model = ResidualizationRegressor(model_constructor, model_params)
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)
model.fit(normalized_X, Y, normalized_Z)

/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


### 5. Discussion and report

In [204]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

if has_batch_effect:
    report_result_from_lmm(model, feature_cols, TRAIT, condition, threshold=0.05, save_output=True,
                           output_dir=OUTPUT_DIR)
else:
    report_result_from_lasso(model, feature_cols, TRAIT, condition, save_output=True, output_dir=OUTPUT_DIR)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0112
Found 100 significant genes associated with the trait 'Thyroid Cancer' conditional on the factor 'Age', with corrected p-value < 0.05:
   Variable  Coefficient  corrected_p_value
     GABRB2     0.199853       9.445009e-24
       LRP4     0.153916       3.454747e-11
    SHROOM4     0.150237       3.783756e-09
      HMGA2     0.122833       4.533386e-08
       EPS8     0.148314       3.922968e-07
    KLHDC8A     0.103442       3.996388e-07
       LIPH     0.126974       5.203673e-07
      MPZL2     0.123835       7.878797e-07
      GABRD     0.085857       3.424810e-06
       GJB3     0.104297       7.705250e-06
   C6orf168     0.106365       1.189886e-05
        FN1     0.142549       1.760688e-05
      KCNQ3     0.132879       2.031323e-05
      GRHL3     0.103693       2.317901e-05
    ZCCHC12     0.091659       3.351467e-05
     ACTBL2     0.094812       6.476432e-05
      SYT12     0.127073       8.24

/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
